In [1]:
from sensitive_info import *

In [2]:
df = pd.read_csv("./CS7_preprocessed.csv")
df.head(2)

,x0,x1,x2,x3,x4,x5,x6,x7,x8,x9,...,x41,x42,x43,x44,x45,x46,x47,x48,x49,y
0,-0.166563,-3.961588,4.621113,2.481908,-1.800135,0.804684,6.718751,-14.789997,-1.040673,-4.204950,...,-1.497117,5.414063,-2.325655,1.674827,-0.264332,60.781427,-7.689696,0.151589,-8.040166,0
1,-0.149894,-0.585676,27.839856,4.152333,6.426802,-2.426943,40.477058,-6.725709,0.896421,0.330165,...,36.292790,4.490915,0.762561,6.526662,1.007927,15.805696,-4.896678,-0.320283,16.719974,0


In [3]:
df = convert_strings_to_category(df)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 160000 entries, 0 to 159999
Data columns (total 51 columns):
 #   Column  Non-Null Count   Dtype   
---  ------  --------------   -----   
 0   x0      159974 non-null  float64 
 1   x1      159975 non-null  float64 
 2   x2      159962 non-null  float64 
 3   x3      159963 non-null  float64 
 4   x4      159974 non-null  float64 
 5   x5      159963 non-null  float64 
 6   x6      159974 non-null  float64 
 7   x7      159973 non-null  float64 
 8   x8      159979 non-null  float64 
 9   x9      159970 non-null  float64 
 10  x10     159957 non-null  float64 
 11  x11     159970 non-null  float64 
 12  x12     159964 non-null  float64 
 13  x13     159969 non-null  float64 
 14  x14     159966 non-null  float64 
 15  x15     159965 non-null  float64 
 16  x16     159974 non-null  float64 
 17  x17     159973 non-null  float64 
 18  x18     159960 non-null  float64 
 19  x19     159965 non-null  float64 
 20  x20     159962 non-null  f

In [4]:
train_df, val_df, test_df = create_train_val_test_sets(df=df, 
                                                       target_column="y", 
                                                       test_size=0.1, 
                                                       val_size=0.1, 
                                                       random_state=7742)

Train Shape: (128000, 51)
Val Shape: (16000, 51)
Test Shape: (16000, 51)


In [5]:
X_train = train_df.drop(columns="y")
y_train = train_df.loc[:,"y"].to_numpy()

X_val = val_df.drop(columns="y")
y_val = val_df.loc[:,"y"].to_numpy()

X_test = test_df.drop(columns="y")
y_test = test_df.loc[:,"y"].to_numpy()

# Subset to features selected by DAI
X_train_dai, X_val_dai, X_test_dai = subset_to_dai_selected_features(X_train=X_train, 
                                                                     X_val=X_val, 
                                                                     X_test=X_test)

DAI Train Shape: (128000, 17)
DAI Val Shape: (16000, 17)
DAI Test Shape: (16000, 17)


In [7]:
scorers_dict = {"accuracy":make_scorer(accuracy_score), 
                "f025_score":make_scorer(fbeta_score, beta=0.25),
                "f05_score":make_scorer(fbeta_score, beta=0.5), 
                "avg_dollars_lost_per_prediction":make_scorer(score_func=average_dollars_scorer_sklearn, 
                                                              greater_is_better=False)}

In [8]:
best_params_lgbm = {"boosting_type":"gbdt",
               "colsample_bytree":0.8,
               "bagging_seed":7742,
               "feature_fraction_seed":7743,
               "importance_type":"gain",
               "learning_rate":0.05,
               "max_bin":63,
               "max_delta_step":0.0,
               "max_depth":10,
               "num_leaves":1024,
               "min_child_samples":20,
               "min_child_weight":0.001,
               "min_data_in_bin":1,
               "min_split_gain":0.0,
               "n_estimators":715,
               "objective":"binary",
               "reg_alpha":0.0,
               "reg_lambda":0.0,
               "scale_pos_weight":1.0,
               "seed":534401655,
               "subsample":0.7,
               "subsample_freq":1}

best_param_grid = {key:[value] for key, value in best_params_lgbm.items()}
best_param_grid_imp = {f"model__{key}":value for key, value in best_param_grid.items()}
best_param_grid_imp['impute__strategy'] = ["mean"]

In [9]:
cv_splitter = StratifiedKFold(n_splits=5, 
                              shuffle=True, 
                              random_state=7742)


train_dai_df = pd.DataFrame(X_train_dai)
train_dai_df['y'] = y_train

train_dai_df.reset_index(drop=True, inplace=True)
X_train_dai = train_dai_df.drop(columns="y")
y_train = train_dai_df.loc[:, "y"]
folds = {}

for index, (train_ind, test_ind) in enumerate(cv_splitter.split(train_dai_df.drop(columns="y"), 
                                                                train_dai_df.loc[:,"y"].to_numpy()), start=1):
    
    folds[f'fold{index}'] = {f"train_ind":train_ind, f"test_ind":test_ind}

In [10]:
lgbm_model_pipe = Pipeline(steps=[("impute", SimpleImputer()), 
                                  ('model', LGBMClassifier(n_jobs=32, random_state=7742))])

In [12]:
all_preds_and_ind = []

In [13]:
folds["fold1"]['test_ind']

array([     0,      7,      8, ..., 127987, 127990, 127995])

## Fold 1 Probabilities from LGBM

In [14]:
model1 = fit_lgbm(X_train=X_train_dai.loc[X_train_dai.index.isin(folds["fold1"]["train_ind"])], 
                  y_train=y_train[folds["fold1"]["train_ind"]].to_numpy(), 
                  X_val=X_train_dai.loc[X_train_dai.index.isin(folds["fold1"]["test_ind"])], 
                  y_val=y_train[folds["fold1"]["test_ind"]].to_numpy(),
                  model_pipe=lgbm_model_pipe, 
                  lgbm_param_grid=best_param_grid_imp, 
                  cv=2)

folds["fold1"]["preds"] = model1.predict_proba(X_train_dai.loc[X_train_dai.index.isin(folds["fold1"]["test_ind"])])[:,1]

[LightGBM] [Warning] seed is set=534401655, random_state=7742 will be ignored. Current value: seed=534401655
Training until validation scores don't improve for 5 rounds
Early stopping, best iteration is:
[436]	valid_0's binary_logloss: 0.131056	valid_0's avg_dollars_lost_per_prediction: 2.5375	valid_0's f05_score: 0.946659
Evaluated only: binary_logloss


## Fold 2 Probabilities from LGBM

In [15]:
model2 = fit_lgbm(X_train=X_train_dai.loc[X_train_dai.index.isin(folds["fold2"]["train_ind"])], 
                  y_train=y_train[folds["fold2"]["train_ind"]].to_numpy(),
                  X_val=X_train_dai.loc[X_train_dai.index.isin(folds["fold2"]["test_ind"])], 
                  y_val=y_train[folds["fold2"]["test_ind"]].to_numpy(),
                  model_pipe=lgbm_model_pipe, 
                  lgbm_param_grid=best_param_grid_imp, 
                  cv=2)

folds["fold2"]["preds"] = model2.predict_proba(X_train_dai.loc[X_train_dai.index.isin(folds["fold2"]["test_ind"])])[:,1]

[LightGBM] [Warning] seed is set=534401655, random_state=7742 will be ignored. Current value: seed=534401655
Training until validation scores don't improve for 5 rounds
Early stopping, best iteration is:
[416]	valid_0's binary_logloss: 0.134026	valid_0's avg_dollars_lost_per_prediction: 2.57656	valid_0's f05_score: 0.945145
Evaluated only: binary_logloss


## Fold 3 Probabilities from LGBM

In [16]:
model3 = fit_lgbm(X_train=X_train_dai.loc[X_train_dai.index.isin(folds["fold3"]["train_ind"])], 
                  y_train=y_train[folds["fold3"]["train_ind"]].to_numpy(), 
                  X_val=X_train_dai.loc[X_train_dai.index.isin(folds["fold3"]["test_ind"])], 
                  y_val=y_train[folds["fold3"]["test_ind"]].to_numpy(),
                  model_pipe=lgbm_model_pipe, 
                  lgbm_param_grid=best_param_grid_imp, 
                  cv=2)

folds["fold3"]["preds"] = model3.predict_proba(X_train_dai.loc[X_train_dai.index.isin(folds["fold3"]["test_ind"])])[:,1]

[LightGBM] [Warning] seed is set=534401655, random_state=7742 will be ignored. Current value: seed=534401655
Training until validation scores don't improve for 5 rounds
Early stopping, best iteration is:
[423]	valid_0's binary_logloss: 0.135549	valid_0's avg_dollars_lost_per_prediction: 2.61094	valid_0's f05_score: 0.944916
Evaluated only: binary_logloss


## Fold 4 Probabilities from LGBM

In [17]:
model4 = fit_lgbm(X_train=X_train_dai.loc[X_train_dai.index.isin(folds["fold4"]["train_ind"])], 
                  y_train=y_train[folds["fold4"]["train_ind"]].to_numpy(), 
                  X_val=X_train_dai.loc[X_train_dai.index.isin(folds["fold4"]["test_ind"])], 
                  y_val=y_train[folds["fold4"]["test_ind"]].to_numpy(),
                  model_pipe=lgbm_model_pipe, 
                  lgbm_param_grid=best_param_grid_imp, 
                  cv=2)

folds["fold4"]["preds"] = model4.predict_proba(X_train_dai.loc[X_train_dai.index.isin(folds["fold4"]["test_ind"])])[:,1]

[LightGBM] [Warning] seed is set=534401655, random_state=7742 will be ignored. Current value: seed=534401655
Training until validation scores don't improve for 5 rounds
Early stopping, best iteration is:
[503]	valid_0's binary_logloss: 0.127151	valid_0's avg_dollars_lost_per_prediction: 2.46563	valid_0's f05_score: 0.947828
Evaluated only: binary_logloss


## Fold 5 Probabilities from LGBM

In [18]:
model5 = fit_lgbm(X_train=X_train_dai.loc[X_train_dai.index.isin(folds["fold5"]["train_ind"])], 
                  y_train=y_train[folds["fold5"]["train_ind"]].to_numpy(), 
                  X_val=X_train_dai.loc[X_train_dai.index.isin(folds["fold5"]["test_ind"])], 
                  y_val=y_train[folds["fold5"]["test_ind"]].to_numpy(),
                  model_pipe=lgbm_model_pipe, 
                  lgbm_param_grid=best_param_grid_imp, 
                  cv=2)

folds["fold5"]["preds"] = model5.predict_proba(X_train_dai.loc[X_train_dai.index.isin(folds["fold5"]["test_ind"])])[:,1]

[LightGBM] [Warning] seed is set=534401655, random_state=7742 will be ignored. Current value: seed=534401655
Training until validation scores don't improve for 5 rounds
Early stopping, best iteration is:
[391]	valid_0's binary_logloss: 0.129679	valid_0's avg_dollars_lost_per_prediction: 2.58516	valid_0's f05_score: 0.945569
Evaluated only: binary_logloss


In [19]:
X_train_dai.head(2)

,x48,x23,x27,x20,x28,x46,x49,x37,x42,x12,x32,x7,x2,x38,x41,x6,x40
0,-4.818280,-4.676670,10.352920,2.029764,-30.102992,32.780607,-14.191745,-72.62,0.582534,-10.735783,0.01,-15.072493,0.419611,-24.254114,-26.823132,0.610082,3.604330
1,3.102613,1.519396,-12.122562,-4.198570,-16.416524,55.582887,34.763313,1918.51,6.188595,9.566781,0.00,2.656567,-0.427442,23.569760,26.066290,-0.621469,-20.189674


In [20]:
X_val_dai.head(2)

,x48,x23,x27,x20,x28,x46,x49,x37,x42,x12,x32,x7,x2,x38,x41,x6,x40
153151,-0.135665,-29.003454,-7.159328,-2.123370,-13.897687,77.038840,28.650379,1438.24,0.964448,24.098276,-0.01,-12.003051,-7.447920,10.262458,11.349467,-10.828716,-8.089228
54081,-0.133721,-4.830448,15.588912,8.475162,4.911999,-52.428648,-17.259751,-1267.47,-6.739039,6.327413,0.00,-73.395886,-10.351719,32.510607,35.954160,-15.050622,-5.562226


In [21]:
X_test_dai.head(2)

,x48,x23,x27,x20,x28,x46,x49,x37,x42,x12,x32,x7,x2,x38,x41,x6,x40
156290,-2.532465,-11.966421,3.949634,10.180605,22.751143,-66.780025,6.334020,-1914.85,7.246131,-37.403131,0.01,44.035923,-1.683509,-0.078549,-0.086869,-2.447696,7.042585
136803,0.108401,4.179954,7.095808,13.903758,21.540864,-57.086176,-3.084239,-1713.34,2.910421,-10.437425,-0.00,-10.607838,-5.937418,10.922252,12.079147,-8.632560,-10.316435


## Validation and Test Set LGBM Probs

In [23]:
final_model = fit_lgbm(X_train=X_train_dai, 
                       y_train=y_train, 
                       X_val=X_val_dai, 
                       y_val=y_val,
                       model_pipe=lgbm_model_pipe, 
                       lgbm_param_grid=best_param_grid_imp, 
                       cv=2)

val_preds = final_model.predict_proba(X_val_dai)[:,1]
X_val_ensemble = X_val_dai.copy(deep=True)
X_val_ensemble['lgbm_probs'] = val_preds


test_preds = final_model.predict_proba(X_test_dai)[:,1]
X_test_ensemble = X_test_dai.copy()
X_test_ensemble["lgbm_probs"] = test_preds

[LightGBM] [Warning] seed is set=534401655, random_state=7742 will be ignored. Current value: seed=534401655
Training until validation scores don't improve for 5 rounds
Early stopping, best iteration is:
[450]	valid_0's binary_logloss: 0.132653	valid_0's avg_dollars_lost_per_prediction: 2.4725	valid_0's f05_score: 0.947468
Evaluated only: binary_logloss


In [24]:
train_inds_and_preds = sorted([(i, p) for i, p in zip([ind for fold in folds.values() for ind in fold['test_ind']], 
                                            [pred for fold in folds.values() for pred in fold['preds']])], key=lambda sublist: sublist[0])
train_inds = [i for i, p in train_inds_and_preds]
train_preds = [p for i, p in train_inds_and_preds]

X_train_ensemble = X_train_dai.copy()
X_train_ensemble["lgbm_probs"] = train_preds

## Fit MLP using LGBM Probs as a features

In [25]:
mlp_pipe = Pipeline(steps=[("impute", SimpleImputer(strategy="mean")),
                           ("model", MLPClassifier(random_state=7742,
                                                   verbose=False,
                                                   warm_start=True,
                                                   early_stopping=True,
                                                   max_iter=20_000, 
                                                   alpha=1e-5, 
                                                   hidden_layer_sizes=(300,200,100), 
                                                   learning_rate_init=0.01))])

mlp_pipe.fit(X_train_ensemble, y_train)

Pipeline(steps=[('impute', SimpleImputer()),
                ('model',
                 MLPClassifier(alpha=1e-05, early_stopping=True,
                               hidden_layer_sizes=(300, 200, 100),
                               learning_rate_init=0.01, max_iter=20000,
                               random_state=7742, warm_start=True))])

In [26]:
X_val_ensemble

,x48,x23,x27,x20,x28,x46,x49,x37,x42,x12,x32,x7,x2,x38,x41,x6,x40,lgbm_probs
153151,-0.135665,-29.003454,-7.159328,-2.123370,-13.897687,77.038840,28.650379,1438.24,0.964448,24.098276,-0.01,-12.003051,-7.447920,10.262458,11.349467,-10.828716,-8.089228,0.002125
54081,-0.133721,-4.830448,15.588912,8.475162,4.911999,-52.428648,-17.259751,-1267.47,-6.739039,6.327413,0.00,-73.395886,-10.351719,32.510607,35.954160,-15.050622,-5.562226,0.999169
16600,2.097463,5.899921,0.172830,-2.465437,-12.928949,-32.265216,-10.913424,553.27,1.851259,-18.125503,0.01,13.303811,7.581987,7.749179,8.569979,11.023639,10.758549,0.007594
54009,-1.276345,11.346091,-1.803787,4.811472,0.621300,-22.999224,-5.638077,350.60,-5.166516,5.728755,-0.01,-44.183957,5.340435,23.432755,25.914774,7.764592,-25.711235,0.007352
40254,-2.327353,-3.466436,-1.026047,-3.795519,6.836663,-37.926008,6.590798,976.14,-3.194539,-29.836625,-0.01,-8.369497,20.411155,34.504885,38.159674,29.676286,4.573880,0.001125
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148571,0.424333,4.823892,-1.424600,5.769466,0.026230,-41.068522,6.130660,-659.39,-4.117836,-9.711528,0.00,-18.526169,-8.615393,12.806808,14.163317,-12.526134,-13.548645,0.869132
110072,2.438535,16.479998,-10.948547,-6.490398,-5.265917,33.477369,23.732092,370.74,1.317636,6.847680,0.01,27.038077,-11.609289,-15.299206,-16.919711,-16.879034,-10.988400,0.017823
46289,-4.075310,-29.923342,-5.067366,-5.965834,-1.562378,-58.700981,-32.546766,329.99,-7.287499,-23.681613,0.00,51.371452,16.875481,-20.724607,-22.919777,24.535682,38.568265,0.822426
4559,-2.681908,-5.122926,1.891111,3.735759,-1.342772,84.641920,17.160410,-596.00,-4.219387,45.799598,-0.01,-47.803456,-25.622991,-18.427550,-20.379413,-37.253903,-28.351833,0.002023


In [27]:
scorers_dict = {"accuracy":make_scorer(accuracy_score), 
                "f025_score":make_scorer(fbeta_score, beta=0.25),
                "f05_score":make_scorer(fbeta_score, beta=0.5), 
                "avg_dollars_lost_per_prediction":make_scorer(score_func=average_dollars_scorer_sklearn, 
                                                              greater_is_better=False)}

evaluate_baseline_model(X_train=X_train_ensemble, 
                        y_train=y_train,
                        X_val=X_val_ensemble, 
                        y_val=y_val, 
                        scorers_dict=scorers_dict,
                        model_pipe=mlp_pipe, 
                        model_name="MLP, LGBM Stacked Ensemble: Validation Set Performance")

========== MLP, LGBM Stacked Ensemble: Validation Set Performance ==========
Training accuracy: 0.959921875
Validation accuracy: 0.9591875

Training f025_score: 0.9481008502226449
Validation f025_score: 0.9491966601592932

Training f05_score: 0.9487653890013731
Validation f05_score: 0.9491757813717243

Training avg_dollars_lost_per_prediction: -2.4846875
Validation avg_dollars_lost_per_prediction: -2.44625



In [28]:
evaluate_baseline_model(X_train=X_train_ensemble, 
                        y_train=y_train,
                        X_val=X_test_ensemble, 
                        y_val=y_test, 
                        model_pipe=mlp_pipe,  
                        scorers_dict=scorers_dict,
                        model_name="MLP, LGBM Stacked Ensemble: Test Set Performance")

========== MLP, LGBM Stacked Ensemble: Test Set Performance ==========
Training accuracy: 0.959921875
Validation accuracy: 0.9595625

Training f025_score: 0.9481008502226449
Validation f025_score: 0.9499174766183753

Training f05_score: 0.9487653890013731
Validation f05_score: 0.9498129675810473

Training avg_dollars_lost_per_prediction: -2.4846875
Validation avg_dollars_lost_per_prediction: -2.41375



In [29]:
train_pred_df = pd.DataFrame({"true":y_train, 
                              "predicted":mlp_pipe.predict(X_train_ensemble)})

In [30]:
# plot_confusion_matrix_from_estimator(df=train_dai_df, 
#                                      target=train_dai_df['y'], 
#                                      model=mlp_pipe)

## Fit MLP using LGBM as features, and scaled inputs

In [32]:
from sklearn.preprocessing import StandardScaler

In [33]:
mlp_pipe_sc = Pipeline(steps=[("impute", SimpleImputer(strategy="mean")), 
                              ("standard_scaler", StandardScaler()),
                           ("model", MLPClassifier(random_state=7742,
                                                   verbose=False,
                                                   warm_start=True,
                                                   early_stopping=True,
                                                   max_iter=20_000, 
                                                   alpha=1e-5, 
                                                   hidden_layer_sizes=(300,200,100), 
                                                   learning_rate_init=0.01))])

mlp_pipe_sc.fit(X_train_ensemble, y_train)

Pipeline(steps=[('impute', SimpleImputer()),
                ('standard_scaler', StandardScaler()),
                ('model',
                 MLPClassifier(alpha=1e-05, early_stopping=True,
                               hidden_layer_sizes=(300, 200, 100),
                               learning_rate_init=0.01, max_iter=20000,
                               random_state=7742, warm_start=True))])

In [34]:
evaluate_baseline_model(X_train=X_train_ensemble, 
                        y_train=y_train,
                        X_val=X_val_ensemble, 
                        y_val=y_val, 
                        scorers_dict=scorers_dict,
                        model_pipe=mlp_pipe_sc, 
                        model_name="MLP, LGBM Stacked Ensemble: Validation Set Performance")

========== MLP, LGBM Stacked Ensemble: Validation Set Performance ==========
Training accuracy: 0.981859375
Validation accuracy: 0.9736875

Training f025_score: 0.9828226619830972
Validation f025_score: 0.9757444383193027

Training f05_score: 0.9810320813956233
Validation f05_score: 0.9728865097716258

Training avg_dollars_lost_per_prediction: -0.8834375
Validation avg_dollars_lost_per_prediction: -1.25125



## No LGBM, No Scaling

In [35]:
mlp_pipe_no_ens = Pipeline(steps=[("impute", SimpleImputer(strategy="mean")),
                           ("model", MLPClassifier(random_state=7742,
                                                   verbose=False,
                                                   warm_start=True,
                                                   early_stopping=True,
                                                   max_iter=20_000, 
                                                   alpha=1e-5, 
                                                   hidden_layer_sizes=(300,200,100), 
                                                   learning_rate_init=0.01))])

mlp_pipe_no_ens.fit(X_train_dai, y_train)

Pipeline(steps=[('impute', SimpleImputer()),
                ('model',
                 MLPClassifier(alpha=1e-05, early_stopping=True,
                               hidden_layer_sizes=(300, 200, 100),
                               learning_rate_init=0.01, max_iter=20000,
                               random_state=7742, warm_start=True))])

In [37]:
evaluate_baseline_model(X_train=X_train_dai, 
                        y_train=y_train,
                        X_val=X_val_dai, 
                        y_val=y_val, 
                        scorers_dict=scorers_dict,
                        model_pipe=mlp_pipe_no_ens, 
                        model_name="MLP (no Scaling): Validation Set Performance")

========== MLP (no Scaling): Validation Set Performance ==========
Training accuracy: 0.9725625
Validation accuracy: 0.9681875

Training f025_score: 0.9665904777640851
Validation f025_score: 0.9643527725107526

Training f05_score: 0.9663296905592087
Validation f05_score: 0.9630059981785636

Training avg_dollars_lost_per_prediction: -1.615625
Validation avg_dollars_lost_per_prediction: -1.75125



## No LGBM, With Scaling

In [38]:
mlp_pipe_sc_no_ens = Pipeline(steps=[("impute", SimpleImputer(strategy="mean")), 
                              ("standard_scaler", StandardScaler()),
                           ("model", MLPClassifier(random_state=7742,
                                                   verbose=False,
                                                   warm_start=True,
                                                   early_stopping=True,
                                                   max_iter=20_000, 
                                                   alpha=1e-5, 
                                                   hidden_layer_sizes=(300,200,100), 
                                                   learning_rate_init=0.01))])

mlp_pipe_sc_no_ens.fit(X_train_dai, y_train)

evaluate_baseline_model(X_train=X_train_dai, 
                        y_train=y_train,
                        X_val=X_val_dai, 
                        y_val=y_val, 
                        scorers_dict=scorers_dict,
                        model_pipe=mlp_pipe_sc_no_ens, 
                        model_name="MLP (with scaling): Validation Set Performance")

========== MLP (with scaling): Validation Set Performance ==========
Training accuracy: 0.98321875
Validation accuracy: 0.9774375

Training f025_score: 0.9806031659200454
Validation f025_score: 0.977706385388908

Training f05_score: 0.9801057589414733
Validation f05_score: 0.9757711333060273

Training avg_dollars_lost_per_prediction: -0.949375
Validation avg_dollars_lost_per_prediction: -1.13125



In [39]:
train_ensemble_df = pd.DataFrame(X_train_ensemble)
train_ensemble_df['y'] = y_train

val_ensemble_df = pd.DataFrame(X_val_ensemble)
val_ensemble_df['y'] = y_val

test_ensemble_df = pd.DataFrame(X_test_ensemble)
test_ensemble_df['y'] = y_test

train_ensemble_df.to_csv("./train_ensemble.csv", index=False)
val_ensemble_df.to_csv("./val_ensemble.csv", index=False)
test_ensemble_df.to_csv("./test_ensemble.csv", index=False)